# 🔗 Record Linkage

- Record Linkage (groupings of various above algorithms)
    - Reference https://en.wikipedia.org/wiki/Record_linkage
    - Python lib https://github.com/J535D165/recordlinkage
    - NLTK Distance references: http://www.nltk.org/_modules/nltk/metrics/distance.html

    
- Jaro-Winkler Distance
    - Reference https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance


- Levenshtein Distance
    - Reference https://en.wikipedia.org/wiki/Levenshtein_distance
    - Python lib https://github.com/ztane/python-Levenshtein


- Gestalt or Ratcliff/Obershelp Pattern Recognition
    - Reference https://en.wikipedia.org/wiki/Gestalt_Pattern_Matching
    - Python lib (SequenceMatcher) https://docs.python.org/3/library/difflib.html 


In [1]:
# Modified from below
# reference: https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html
# note: The data used in this example is part of Febrl and is fictitious.
import pandas as pd
import recordlinkage
from recordlinkage.datasets import load_febrl4

dfA, dfB = load_febrl4()

dfA.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625


In [2]:
dfB.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999


In [3]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)

In [4]:
indexer = recordlinkage.Index()
indexer.block("given_name")
candidate_links = indexer.index(dfA, dfB)

print(len(candidate_links))

77249


In [5]:
compare_cl = recordlinkage.Compare()
compare_cl.string("suburb", "suburb", method="jarowinkler", label="suburb")

features = compare_cl.compute(candidate_links, dfA, dfB)
features.sort_values(by="suburb", ascending=False).head(5)

,,suburb
rec_id_1,rec_id_2,
rec-4224-org,rec-4224-dup-0,1.0
rec-354-org,rec-354-dup-0,1.0
rec-2813-org,rec-2813-dup-0,1.0
rec-2747-org,rec-2747-dup-0,1.0
rec-1694-org,rec-1694-dup-0,1.0


In [6]:
features = features.rename(columns={"suburb": "suburb_match"})
features = features.reset_index(drop=False)

features_with_record = pd.merge(
    features,
    dfA.rename(columns={"suburb": "suburb_A"})["suburb_A"],
    how="left",
    left_on="rec_id_1",
    right_on=dfA.index,
)
features_with_record

,rec_id_1,rec_id_2,suburb_match,suburb_A
0,rec-1070-org,rec-3024-dup-0,0.584046,winston hills
1,rec-1070-org,rec-2371-dup-0,0.410256,winston hills
2,rec-1070-org,rec-4652-dup-0,0.515797,winston hills
3,rec-1070-org,rec-4795-dup-0,0.458689,winston hills
4,rec-1070-org,rec-1314-dup-0,0.313187,winston hills
...,...,...,...,...
77244,rec-4528-org,rec-4528-dup-0,1.000000,swan hill
77245,rec-4887-org,rec-4887-dup-0,0.983333,mooroolbark
77246,rec-4350-org,rec-4350-dup-0,1.000000,kuraby
77247,rec-4569-org,rec-4569-dup-0,1.000000,st peters


In [7]:
features_with_record = pd.merge(
    features_with_record,
    dfB.rename(columns={"suburb": "suburb_B"})["suburb_B"],
    how="left",
    left_on="rec_id_2",
    right_on=dfB.index,
)
features_with_record

,rec_id_1,rec_id_2,suburb_match,suburb_A,suburb_B
0,rec-1070-org,rec-3024-dup-0,0.584046,winston hills,jamdstown
1,rec-1070-org,rec-2371-dup-0,0.410256,winston hills,woodcroft
2,rec-1070-org,rec-4652-dup-0,0.515797,winston hills,avondale hdights
3,rec-1070-org,rec-4795-dup-0,0.458689,winston hills,coombabah
4,rec-1070-org,rec-1314-dup-0,0.313187,winston hills,rosebud
...,...,...,...,...,...
77244,rec-4528-org,rec-4528-dup-0,1.000000,swan hill,swan hill
77245,rec-4887-org,rec-4887-dup-0,0.983333,mooroolbark,mooroolbuark
77246,rec-4350-org,rec-4350-dup-0,1.000000,kuraby,kuraby
77247,rec-4569-org,rec-4569-dup-0,1.000000,st peters,st peters


In [8]:
features_with_record[["suburb_match", "suburb_A", "suburb_B"]].sort_values(
    by="suburb_match", ascending=False
).head(5)

,suburb_match,suburb_A,suburb_B
63087,1.0,moree,moree
17067,1.0,marsden,marsden
34279,1.0,fullarton,fullarton
8629,1.0,mirboo north,mirboo north
75918,1.0,broadmeadows,broadmeadows


In [9]:
features_with_record[
    (features_with_record["suburb_match"] != 0)
    & (features_with_record["suburb_match"] != 1)
][["suburb_match", "suburb_A", "suburb_B"]].sort_values(
    by="suburb_match", ascending=False
).head(
    20
)

,suburb_match,suburb_A,suburb_B
76050,0.988889,narre warren north,narre wraren north
21937,0.988889,brighton-le-sands,brighton-l e-sands
72947,0.988889,birmingham gardens,birminghma gardens
72576,0.987500,surfers paradise,surfersp aradise
52988,0.987500,pacific paradise,pacificparadise
75638,0.987500,hoppers crossing,hoppers corssing
66891,0.987500,st georges basin,st george sbasin
63924,0.987500,hoppers crossing,hoppersc rossing
50116,0.987500,shoalhaven heads,shoalhave nheads
64465,0.987500,launceston south,launcestno south
